## Problem 1
Perform an appropriate train-test split on the data. In your report, justify why <br>
you chose to do the split in the way you did. In further problems, only ever <br>
perform exploratory data analysis on the training data, as testing data should <br>
never be used to draw hypotheses.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

RANDOM_STATE = 1

raw_df = pd.read_csv("life_expectancy.csv")
# Test size of 0.25 because this was recommended in a lecture.
train_df: pd.DataFrame
test_df: pd.DataFrame
train_df, test_df = train_test_split(raw_df, test_size=0.25, random_state=RANDOM_STATE)

# Calculate the Pearson correlation coefficient between each feature and the target LEB under column "Life Expectancy at Birth, both sexes (years)"
country_LEB_pearson = train_df.corr("pearson")


ValueError: could not convert string to float: 'Zambia'